In [9]:
# Test script to see if scrape/stack.py works
import os
# from dotenv import load_dotenv
from typing import Generator, List, Dict, Union, Tuple
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras as k
# CategoryEncoding - https://keras.io/api/layers/preprocessing_layers/categorical/category_encoding/#categoryencoding-class
from tensorflow.keras import layers
import pandas as pd
from dotenv import load_dotenv
# from sklearn.model_selection import train_test_split
from scrape.types import StackOverflowAnswer, is_stackoverflow_answer
from util.db import Database
from transform.snippet import SnippetLexer, LexerStatus

In [10]:
load_dotenv()
print(tf.test.gpu_device_name())
db = Database()

/device:GPU:0


2021-12-15 23:10:01.046268: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-15 23:10:01.046289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# The number of tokens that must be included in each feature vector. Snippets
# with more tokens will be truncated, and snippets with fewer tokens will be
# padded with zeros. (right-padded)
toks_per_seq: int = 2500

# The dimensionality of the LSTM's hidden state. This is the size of the
# output(final) vector
hidden_layer_size: int = 500

In [12]:
# Load in question and answer data
dataset_size = 1000
answers = pd.DataFrame.from_records(list(db.answers.find().limit(dataset_size)), index='_id')
questions = pd.DataFrame.from_records(list(db.questions.find().limit(dataset_size)), index='_id')

print(answers.info())
print(questions.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 70142619 to 34291984
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   answer_id                             1000 non-null   int64  
 1   author_id                             990 non-null    float64
 2   author_username                       1000 non-null   object 
 3   is_accepted                           1000 non-null   bool   
 4   is_highest_scored                     1000 non-null   bool   
 5   page_pos                              1000 non-null   int64  
 6   question_has_highest_accepted_answer  1000 non-null   bool   
 7   question_id                           1000 non-null   int64  
 8   score                                 1000 non-null   int64  
 9   snippets                              1000 non-null   object 
 10  source                                1000 non-null   object 
dtypes: boo

In [13]:
# Clean up the answers a lil bit
answers['snippets'] = answers['snippets'].str.strip()
answers.head(2)

,answer_id,author_id,author_username,is_accepted,is_highest_scored,page_pos,question_has_highest_accepted_answer,question_id,score,snippets,source
_id,,,,,,,,,,,
70142619,70142619,4593267.0,chqrlie,False,True,1,False,70142439,1,*list = new_node;,https://stackoverflow.com/a/70142619
70142654,70142654,17067764.0,misfit,False,False,2,False,70142439,0,(*list) = new_node;,https://stackoverflow.com/a/70142654


In [14]:
# Clean the questions up a lil bit, and remove columns we won't need
questions.drop(columns=['content_license', 'creation_date', 'last_activity_date', 'last_edit_date', 'owner', 'protected_date', 'community_owned_date', 'migrated_from', 'locked_date'], inplace=True, errors='ignore')
questions.head(2)

,answer_count,is_answered,link,question_id,score,tags,title,view_count,accepted_answer_id,closed_date,closed_reason
_id,,,,,,,,,,,
70142439,2,True,https://stackoverflow.com/questions/70142439/l...,70142439,1,"[arrays, c, pointers, struct, linked-list]",linked list not inserting new values &quot;C l...,41,NaN,NaN,NaN
70141741,1,True,https://stackoverflow.com/questions/70141741/t...,70141741,2,"[c, dynamic-memory-allocation, trim, c-strings...",trim function halve the memory size to remove ...,40,70141881.0,NaN,NaN


In [15]:
# Join the questions and answers together on the question id
df = answers.merge(questions[['score', 'title', 'view_count', 'closed_date', 'closed_reason']], how='inner', left_on='question_id', right_on='_id', suffixes=('_answer', '_question'))
df.head()

,answer_id,author_id,author_username,is_accepted,is_highest_scored,page_pos,question_has_highest_accepted_answer,question_id,score_answer,snippets,source,score_question,title,view_count,closed_date,closed_reason
0,70142619,4593267.0,chqrlie,False,True,1,False,70142439,1,*list = new_node;,https://stackoverflow.com/a/70142619,1,linked list not inserting new values &quot;C l...,41,NaN,NaN
1,70142654,17067764.0,misfit,False,False,2,False,70142439,0,(*list) = new_node;,https://stackoverflow.com/a/70142654,1,linked list not inserting new values &quot;C l...,41,NaN,NaN
2,3501681,2411320.0,gsamaras,False,True,1,False,3501338,352,#define _GNU_SOURCE\n#include <stdio.h>\n#incl...,https://stackoverflow.com/a/3501681,231,C read file line by line,1024701,NaN,NaN
3,3502293,169346.0,jeremyp,False,False,8,False,3501338,4,lineBuffer[count] = '\0';\n\nchar buffer [BUFF...,https://stackoverflow.com/a/3502293,231,C read file line by line,1024701,NaN,NaN
4,3501425,387076.0,gilles-so-stop-being-evil,False,False,3,False,3501338,23,lineBuffer[count] = '\0';\n realloc(lineBuf...,https://stackoverflow.com/a/3501425,231,C read file line by line,1024701,NaN,NaN


In [16]:
# Temporarily truncate the dataset to only include the first 1000 answers to
# make development faster
# df = df.head(1000)

In [17]:
# This is OK because questions with zero score were filtered out during scraping
df['score_ratio'] = df['score_answer'] / df['score_question']
score_ratio_min: float = df['score_ratio'].min()
score_ratio_max: float = df['score_ratio'].max()
score_ratio_diff = score_ratio_max - score_ratio_min

# Normalize score ratio between -1 and 1
df['score_ratio_norm'] = (2 * (df['score_ratio'] - score_ratio_min) / score_ratio_diff) - 1

df.head()

,answer_id,author_id,author_username,is_accepted,is_highest_scored,page_pos,question_has_highest_accepted_answer,question_id,score_answer,snippets,source,score_question,title,view_count,closed_date,closed_reason,score_ratio,score_ratio_norm
0,70142619,4593267.0,chqrlie,False,True,1,False,70142439,1,*list = new_node;,https://stackoverflow.com/a/70142619,1,linked list not inserting new values &quot;C l...,41,NaN,NaN,1.000000,-0.500000
1,70142654,17067764.0,misfit,False,False,2,False,70142439,0,(*list) = new_node;,https://stackoverflow.com/a/70142654,1,linked list not inserting new values &quot;C l...,41,NaN,NaN,0.000000,-0.750000
2,3501681,2411320.0,gsamaras,False,True,1,False,3501338,352,#define _GNU_SOURCE\n#include <stdio.h>\n#incl...,https://stackoverflow.com/a/3501681,231,C read file line by line,1024701,NaN,NaN,1.523810,-0.369048
3,3502293,169346.0,jeremyp,False,False,8,False,3501338,4,lineBuffer[count] = '\0';\n\nchar buffer [BUFF...,https://stackoverflow.com/a/3502293,231,C read file line by line,1024701,NaN,NaN,0.017316,-0.745671
4,3501425,387076.0,gilles-so-stop-being-evil,False,False,3,False,3501338,23,lineBuffer[count] = '\0';\n realloc(lineBuf...,https://stackoverflow.com/a/3501425,231,C read file line by line,1024701,NaN,NaN,0.099567,-0.725108


In [18]:
# Turn the code snippets into tokens
lexer = SnippetLexer()

def tokenize(snippet: str) -> Tuple[List, LexerStatus]:
  toks, status = lexer.lex(snippet)
  toks = np.array([tok.type for tok in toks], dtype=str)

  # Right pad the tokens util they are toks_per_seq long
  pad_amount = toks_per_seq - len(toks)
  if pad_amount > 0:
    toks = np.pad(toks, (0, pad_amount), 'constant', constant_values='<PAD>')

  return toks, status

df[['tokens', 'lexer_status']] = pd.DataFrame(df['snippets'].apply(tokenize).tolist(), index=df.index)
df.head()


,answer_id,author_id,author_username,is_accepted,is_highest_scored,page_pos,question_has_highest_accepted_answer,question_id,score_answer,snippets,source,score_question,title,view_count,closed_date,closed_reason,score_ratio,score_ratio_norm,tokens,lexer_status
0,70142619,4593267.0,chqrlie,False,True,1,False,70142439,1,*list = new_node;,https://stackoverflow.com/a/70142619,1,linked list not inserting new values &quot;C l...,41,NaN,NaN,1.000000,-0.500000,"[TIMES, TYPEID, EQUALS, TYPEID, SEMI, <PAD>, <...",success
1,70142654,17067764.0,misfit,False,False,2,False,70142439,0,(*list) = new_node;,https://stackoverflow.com/a/70142654,1,linked list not inserting new values &quot;C l...,41,NaN,NaN,0.000000,-0.750000,"[LPAREN, TIMES, TYPEID, RPAREN, EQUALS, TYPEID...",success
2,3501681,2411320.0,gsamaras,False,True,1,False,3501338,352,#define _GNU_SOURCE\n#include <stdio.h>\n#incl...,https://stackoverflow.com/a/3501681,231,C read file line by line,1024701,NaN,NaN,1.523810,-0.369048,"[PPHASH, TYPEID, TYPEID, PPHASH, TYPEID, LT, T...",success
3,3502293,169346.0,jeremyp,False,False,8,False,3501338,4,lineBuffer[count] = '\0';\n\nchar buffer [BUFF...,https://stackoverflow.com/a/3502293,231,C read file line by line,1024701,NaN,NaN,0.017316,-0.745671,"[TYPEID, LBRACKET, TYPEID, RBRACKET, EQUALS, C...",success
4,3501425,387076.0,gilles-so-stop-being-evil,False,False,3,False,3501338,23,lineBuffer[count] = '\0';\n realloc(lineBuf...,https://stackoverflow.com/a/3501425,231,C read file line by line,1024701,NaN,NaN,0.099567,-0.725108,"[TYPEID, LBRACKET, TYPEID, RBRACKET, EQUALS, C...",error


In [19]:
# Remove snippets that failed to tokenize
df_start_len = len(df)
df = df[df['lexer_status'] == 'success']
df_filtered_len = len(df)
removed_percent = (df_start_len - df_filtered_len) / df_start_len * 100
print(f'Filtered out {removed_percent:.2f}% of the dataset because they failed tokenization')

Filtered out 18.50% of the dataset because they failed tokenization


In [20]:
lexer = SnippetLexer()
dev_null = open(os.devnull, 'w')

def get_corpus():
    corpus: set[str] = set()
    for tok_list in df['tokens']:
        # assert type(tok_list) == list, f'Expected list, got {type(tok_list)}'
        assert len(tok_list) == toks_per_seq
        for tok in tok_list:
            corpus.add(tok)

    return np.array(list(corpus))

corpus = get_corpus()
corpus_len = len(corpus)
print(f'corpus has {corpus_len} token types')
corpus_dict: Dict[str, int] = dict()

for i, type in enumerate(corpus):
    corpus_dict[type] = i

print(corpus)

corpus has 98 token types
['IF' 'OREQUAL' 'RESTRICT' 'SIGNED' 'EXTERN' 'DOUBLE' 'PPPRAGMASTR'
 'PPPRAGMA' 'RBRACE' 'MINUS' 'DIVIDE' 'OFFSETOF' 'GT' 'FOR' 'ANDEQUAL'
 'BREAK' 'AND' 'COMMA' 'PERIOD' 'EQUALS' 'AUTO' 'COLON' 'TIMES' 'DEFAULT'
 'XOR' 'UNSIGNED' 'ARROW' '__INT128' 'SIZEOF' 'FLOAT_CONST' 'LONG'
 'VOLATILE' 'STATIC' 'MINUSMINUS' '<PAD>' 'LT' 'RPAREN' 'TYPEDEF' 'GOTO'
 'FLOAT' 'UNION' 'OR' 'RSHIFT' 'LOR' 'SEMI' 'INLINE' 'LE' 'REGISTER'
 'LBRACKET' 'DO' 'PLUSEQUAL' 'TIMESEQUAL' 'RETURN' 'CONDOP' 'PPHASH'
 'LAND' 'LBRACE' '_BOOL' 'MODEQUAL' 'PLUS' 'EQ' 'MOD' 'NE' 'U32CHAR_CONST'
 'STRUCT' 'CONTINUE' 'INT_CONST_BIN' 'VOID' 'ELLIPSIS' 'INT_CONST_DEC'
 'PLUSPLUS' 'LSHIFT' 'NOT' 'LNOT' 'STRING_LITERAL' 'RBRACKET' 'CHAR_CONST'
 'SHORT' 'WHILE' 'CASE' 'HEX_FLOAT_CONST' 'CONST' 'INT' 'LPAREN'
 'RSHIFTEQUAL' 'INT_CONST_CHAR' 'XOREQUAL' 'ENUM' 'DIVEQUAL' 'LSHIFTEQUAL'
 'TYPEID' 'MINUSEQUAL' 'ELSE' 'INT_CONST_HEX' 'SWITCH' 'CHAR'
 'INT_CONST_OCT' 'GE']


In [21]:
# https://keras.io/api/layers/preprocessing_layers/categorical/category_encoding/#categoryencoding-class
category_layer = layers.CategoryEncoding(num_tokens=corpus_len, output_mode='one_hot', name='token-encoder')

# def toks_to_int(toks: List[str]) -> List[int]:
#     np.asarray([corpus_dict[tok] for tok in toks])
def tok_to_int(tok: str) -> int:
    return corpus_dict[tok]
toks_to_ints = np.vectorize(tok_to_int)
# vtoks_to_int = np.vectorize(toks_to_int)
df['tokens'] = df['tokens'].apply(toks_to_ints).apply(category_layer).apply(lambda x: x.numpy())
df.head()

2021-12-15 23:10:04.124797: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-15 23:10:04.124820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


,answer_id,author_id,author_username,is_accepted,is_highest_scored,page_pos,question_has_highest_accepted_answer,question_id,score_answer,snippets,source,score_question,title,view_count,closed_date,closed_reason,score_ratio,score_ratio_norm,tokens,lexer_status
0,70142619,4593267.0,chqrlie,False,True,1,False,70142439,1,*list = new_node;,https://stackoverflow.com/a/70142619,1,linked list not inserting new values &quot;C l...,41,NaN,NaN,1.000000,-0.500000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",success
1,70142654,17067764.0,misfit,False,False,2,False,70142439,0,(*list) = new_node;,https://stackoverflow.com/a/70142654,1,linked list not inserting new values &quot;C l...,41,NaN,NaN,0.000000,-0.750000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",success
2,3501681,2411320.0,gsamaras,False,True,1,False,3501338,352,#define _GNU_SOURCE\n#include <stdio.h>\n#incl...,https://stackoverflow.com/a/3501681,231,C read file line by line,1024701,NaN,NaN,1.523810,-0.369048,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",success
3,3502293,169346.0,jeremyp,False,False,8,False,3501338,4,lineBuffer[count] = '\0';\n\nchar buffer [BUFF...,https://stackoverflow.com/a/3502293,231,C read file line by line,1024701,NaN,NaN,0.017316,-0.745671,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",success
5,15626701,1017417.0,lefteris-e,False,False,16,False,3501338,0,const char* func x(){\n char line[100];\n ...,https://stackoverflow.com/a/15626701,231,C read file line by line,1024701,NaN,NaN,0.000000,-0.750000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",success


In [26]:
# tok_preprocessor = layers.CategoryEncoding(num_tokens=corpus_len, output_mode='one_hot')(inputs)

model: k.Sequential = keras.Sequential([
    # layers.Input(shape=(toks_per_seq, corpus_len)), # TODO
    layers.LSTM(hidden_layer_size, return_sequences=False, name='tok-lstm', input_shape=(toks_per_seq, corpus_len)),
    # TODO: If 1 layer doesn't do it, uncomment dis boi
    # layers.Dense(corpus_len, activation='relu', name='post-lstm-1'),
    layers.Dense(1, name='output')
])
# model(encoded)
# model.build((toks_per_seq, corpus_len))
model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tok-lstm (LSTM)             (None, 500)               1198000   
                                                                 
 output (Dense)              (None, 1)                 501       
                                                                 
Total params: 1,198,501
Trainable params: 1,198,501
Non-trainable params: 0
_________________________________________________________________


In [27]:
# df_permutated = df.sample(frac=1)

# train_size = 0.8
# train_end = int(len(df_permutated) * train_size)

# df_train = df_permutated[:train_end]
# df_test = df_permutated[train_end:]

num_epochs = 10
# model.fit(df_train['tokens'], df_train['score_ratio_norm'], epochs=num_epochs)
model.fit(np.asarray(df['tokens']), np.asarray(df['score_ratio_norm']), epochs=num_epochs, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).